# Add charges to metabolites without charge

Note: This notebook is obsolete since a better result can now be achieved with the [MCC tool](https://github.com/Biomathsys/MassChargeCuration).

## 1. From *P. putida*

We noticed that many metabolites had no denoted charge at all. After going through a list of these metabolites, we saw that a lot of them were also present in the model of *Pseudomonas putida* `iJN1463`. 

In [ ]:
from cobra.io.sbml import validate_sbml_model
from refinegems.io import load_model_libsbml
from libsbml import writeSBMLToFile

In [ ]:
example_path = '../../../../data/iJN1463.xml'
example_model = load_model_libsbml(example_path)
print(example_model.getId())

modelpaths_to_change = ['../../../../models/iCstr1054FB23.xml', '../../../../models/iCstr1197FB23.xml', '../../../../models/iCstr1115FB23.xml', '../../../../models/iCstr1116FB23.xml']

for path_to_change in modelpaths_to_change:
    model_to_change = load_model_libsbml(path_to_change)
    spe = model_to_change.getListOfSpecies()
    uncharged_to_change = []
    
    for i in spe:
        bigg = i.getId()[2:-2]
        if not i.getPlugin('fbc').isSetCharge(): # we are only interested in metab without charge
            if bigg not in uncharged_to_change:
                uncharged_to_change.append(bigg)

    for metab in uncharged_to_change:
        compart = ['_c', '_p', '_e']
        for comp in compart:
            try:
                met = example_model.getSpecies('M_' + metab + comp)
                charge = met.getPlugin('fbc').getCharge()
                print('M_' + metab + comp, charge)
                met_to_change = model_to_change.getSpecies('M_' + metab + comp)
                met_to_change.getPlugin('fbc').setCharge(charge)
            except (AttributeError):
                print('M_' + metab + comp + ' does not exist in P. putida.')


    writeSBMLToFile(model_to_change.getSBMLDocument(),path_to_change)
    print("Polished model written to " + path_to_change)
    test, errors = validate_sbml_model(path_to_change)
    print(errors)

Next we wanted to check how many metabolites were still without charge. 

Note: If you execute this notebook later this will most likely lead to no output since by now all metabolites have a denoted charge.

In [ ]:
def get_metab_without_charge(metab_list):
    uncharged = []
    for i in spe:
        bigg = i.getId()[2:-2]
        if not i.getPlugin('fbc').isSetCharge(): # we are only interested in metab without charge
            if bigg not in uncharged:
                uncharged.append(bigg)
    return uncharged

In [ ]:
path = '../../../../models/iCstr1054FB23.xml'
model = load_model_libsbml(path)
print(get_metab_without_charge(model.getListOfSpecies()))

## 2. From researching manually

We took the lists generated by the function above and searched on CHEBI manually for charges. The result of this is denoted below.

In [ ]:
zero = ['actn__S' , 'ala_L_his__L', 'ala_L_leu__L', 'alagly', '4crsol', 'stfrnB', 'stfrnA', 'hmbpp', 'gly_phe__L', 'glyglygln', 'gly_tyr__L', 'gly_met__L', 'gly_leu__L', 'gly_cys__L', 'abg4', 'istfrnA', 'lysglugly', 'prohisglu', 'serglugly', 'nacg', 'ala_L_gln__L', 'abt__L', 'm2bcoa', 'ctncoa', 'pppgpp', '1ag160' , '1ag180', '1ag181d9', '1ag182d9d12', 'ala_L_thr__L', 'ala_L_gln__L', 'gly_gln__L', 'ala_L_Thr__L', 'nh3', 'tag160', 'dag181d9', 'salchs4', 'gly_pro__L', 'gly_asn__L', 'abg4', 'istfrnA', '2m35mdntha', '35dnta', 'metox']
minus_1 = ['2ahbut', 'ala_L_glu__L', 'pa160190', 'pa190190', 'lgt__S', 'hethmpp', 'met_L_ala__L', '2ahbut', '2hetdp', '3h4atb', '3hasp__L', 'R_3hdcaa', 'mmalsa__S', 'op4en', '6atha', 'gly_glu__L', 'gly_asp__L', '2hadnt', '4hadnt', '1p2cbxl']
minus_2 = ['nadhx__R', 'nadhx__S', 'istfrnB', 'dhpmp', '4h2kpi', '4abzglu']
minus_4 = ['ddcoa', 'nadphx__R', 'nadphx__S', 'mhpglu', 'tagdp__D', '2oxpaccoa', '3h4atbcoa', '3h6athcoa', '3hbycoa', '3hdd5coa', '3hdd6coa', '3hddccoa', '3hdec4coa', '3hhd58coa', '3hhdccoa', '3hhpcoa', '3hnonacoa', 'dd2coa', 'tded5_2_coa', '6athacoa', '3hpbcoa', '3hpdecacoa', '3hphpcoa', '3hphxacoa', '3hpnonacoa', '3hpoctacoa', '6ath2coa', '3hocoa', 'decoa']
minus_5 = ['23dhacoa']
minus_7 = ['dscl']
plus_3 = ['salchs4fe']

charges = {0: zero, -1 : minus_1, -2 : minus_2, -4: minus_4, -5 : minus_5, 
           -7 : minus_7, 3: plus_3}

In [ ]:
def key_return(X):
    for key, value in charges.items():
        if X == value:
            return key
        if isinstance(value, list) and X in value:
            return key
    return 1000 # set to 1000 if no charge could be determined

In [ ]:
modelpaths = ['../../models/iCstr1054FB23.xml', '../../models/iCstr1197FB23.xml', '../../models/iCstr1115FB23.xml', '../../models/iCstr1116FB23.xml']

for path in modelpaths:
    model = load_model_libsbml(path)
    spe = model.getListOfSpecies()

    for metabolite in spe:
        #if not i.getPlugin('fbc').isSetCharge(): # we are only interested in metab without charge
        bigg = metabolite.getId()[2:-2]
        charge = key_return(bigg)
        if charge < 1000:
            print(bigg, charge)
            metabolite.getPlugin('fbc').setCharge(int(charge))

    writeSBMLToFile(model.getSBMLDocument(),path)
    print("Polished model written to " + path)
    test, errors = validate_sbml_model(path)
    print(errors)